In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pywt
import pandas as pd
import os
import time
from scipy import signal
from stingray import lightcurve
import sys
from stingray import Bispectrum
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")
c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:40: UserWarning: Numba not installed. Faking it
  warnings.warn("Numba not installed. Faking it")


In [2]:
# file that will be skipped because of information loss
zeros_test = ['co2a0000368_91.csv', 'co2c0000341_26.csv']
zeros_train = ['co2a0000368_0.csv', 'co2a0000368_1.csv', 'co2a0000368_2.csv', 'co2a0000368_3.csv', 'co2a0000368_4.csv', 'co2a0000368_5.csv', 'co2c0000341_27.csv']

### Menghitung Matriks Cumulant orde ke-3

In [3]:
def calcCumulantOrde3(df_data, t, lag):
    # Compute the bispectrum of the signal
    lc = lightcurve.Lightcurve(t,df_data.T)
    bs = Bispectrum(lc, maxlag=lag)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return bs.bispec_mag

### Creating Filter Matrix

In [4]:
def gkernel(k=2, l=1, sig=20):
    """
    Gaussian Kernel Creator via given length and sigma
    """

    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    bx = np.linspace(-(k - 1) / 2., (k - 1) / 2., k)

    xx, yy = np.meshgrid(ax, bx)

    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sig))

    return kernel / np.max(kernel)

### Creating feature matrix

In [11]:
def create_feature(bispectrum, pyramid, fil_size):
    fsize = len(fil_size)
    feature_matrix = np.zeros([fsize,fsize])
    xtrack = 0
    ytrack = 0
    for xdim in range(fsize):
        for ydim in range(fsize):
            x,y = fil_size[xdim], fil_size[ydim]
            # print(xtrack, ytrack)
            
            feature_matrix[xdim][ydim] = np.mean(bispectrum[xtrack:xtrack+x, ytrack:ytrack+y] * pyramid[xtrack:xtrack+x, ytrack:ytrack+y])
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    final_feature = feature_matrix[np.triu(np.ones_like(feature_matrix, dtype=bool))]
    final_feature = final_feature[~np.isnan(final_feature)]

    return feature_matrix.flatten()

    # x, y = np.meshgrid(np.arange(256), np.arange(256))
    # z = overall
    # z = z[:-1, :-1]
    # z_min, z_max = -np.abs(z).max(), np.abs(z).max()

    # plt.subplot()

    # plt.pcolormesh(x, y, z, 
    #             cmap =cm.coolwarm, 
    #             vmin = z_min, 
    #             vmax = z_max,
    #             edgecolors = 'face',
    #             shading ='flat')

    # plt.title('Filter')

    # # set the limits of the plot
    # # to the limits of the data
    # plt.axis([x.min(), x.max(), y.min(), y.max()])

    # plt.colorbar()
    # plt.show()

    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 5))
    # ax = plt.axes(projection="3d")
    # x2d, y2d = np.meshgrid(np.arange(256), np.arange(256))
    # ax.plot_surface(x2d, y2d, overall, cmap=cm.coolwarm,
    #                         linewidth=0, antialiased=False)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('height')
    # plt.show()

### Persiapan data

In [9]:
# Define sampling frequency
fs = 256
t = np.arange(0, 1, 1/fs)

def get_csv_EEG(filename):
    # Load data from CSV
    data = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=range(3,259))
    channel_name = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')
    
    df_data = pd.DataFrame(data.T, columns=channel_name)

    df_data = df_data.drop(columns=['X', 'Y', 'nd'])

    return df_data, df_data.columns


### Perhitungan Feature Bispectrum

In [7]:
def extract_feature(directory, lag, filter_dim = 5):
    bis_dim = lag
    big_fil_size = bis_dim //2
    sigma = bis_dim//8
    if sigma == 0:
        sigma = 1
    fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
    
    if sum(fil_size) != bis_dim:
            fil_size = [value for value in fil_size if value != 0]
            if len(fil_size) == 1:
                fil_size.append(fil_size[0])
    
    fil_size[-1] = fil_size[-2] 
    pyramid = np.zeros([bis_dim,bis_dim])
    xtrack, ytrack = 0,0
    print(fil_size)
    for xdim in range(len(fil_size)):
        for ydim in range(len(fil_size)):
            x,y = fil_size[xdim], fil_size[ydim]
            pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = gkernel(x, y, sigma)
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum2dd')+"_" + str(lag),foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum2dd' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcCumulantOrde3(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature)
                bispectrum = np.array(bispectrum)
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum2d'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### Test

In [ ]:
def extract_test(filename, lag, pyramid, fil_size):
    
    if filename in zeros_train or filename in zeros_test:
        pass
    if 'metadata' in filename.lower():
        pass
    
    df_data, channel_name = get_csv_EEG(filename)
    bispectrum = []
    for channel in channel_name:
        magnitude = calcCumulantOrde3(df_data[channel], t, lag)
        feature = create_feature(magnitude,pyramid,fil_size)
        bispectrum.append(feature)
    bispectrum = np.array(bispectrum)
    bispectrum = bispectrum.flatten()
    return bispectrum
                


In [ ]:
bis_dim = 256
filter_dim = 5
big_fil_size = bis_dim //2
fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
fil_size[-1] = fil_size[-2] 
pyramid = np.zeros([bis_dim,bis_dim])
xtrack, ytrack = 0,0
for xdim in range(filter_dim):
    for ydim in range(filter_dim):
        x,y = fil_size[xdim], fil_size[ydim]
        pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = create_pyramid(x,y)
        ytrack = ytrack+y
    ytrack = 0
    xtrack = xtrack+x
bs = extract_test('../SMNI_CMI_TEST_CSV\co2a0000364\co2a0000364_30.csv', 256, pyramid, fil_size)
bs.shape

### Main Program

In [10]:
lags = [256, 128, 64, 32, 16, 8, 4, 2]
for lag in lags:
    extract_feature('../SMNI_CMI_TEST_CSV', lag)
    extract_feature('../SMNI_CMI_TRAIN_CSV', lag)

KeyboardInterrupt: 

In [ ]:
data = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=range(3,259))
channel_name = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')

df_data = pd.DataFrame(data.T, columns=channel_name)
df_data = df_data.drop(columns=['X', 'Y', 'nd'])
bispectrum = []
for channel in df_data.columns:
    magnitude = calcCumulantOrde3(df_data[channel], t, 256)
    bispectrum.append(magnitude)
bispectrum = np.array(bispectrum)
np.save('../bispectrum.npy', bispectrum)
x = np.load('../bispectrum.npy')

(bispectrum == x).all()